In [ ]:
# default_exp testing_utils
from nbdev.showdoc import *
import numpy as np
import matplotlib.pyplot as plt
import torch
import FRED
if torch.__version__[:4] == '1.14': # If using pytorch with MPS, use Apple silicon GPU acceleration
    device = torch.device("cuda" if torch.cuda.is_available() else 'mps' if torch.has_mps else "cpu")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", device)
%load_ext autoreload
%autoreload 2

Using device cpu
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Comprehensive PHATE Comparisons on Single Cell Datasets

Explanation: this notebook prepares a `papermill` powered test of the toy dataset benchmarker, using the noisy double helix as our protege.

Here's what it does:
1. Start with a dict of possible parameters, each contained in a list, like this:

In [ ]:
import papermill as pm

In [ ]:
parameters_dict = {
    'notebook' : ['05d02 PHATE Comparison on sc.ipynb'], # the notebook we are inserting these parameters into
    'dataset_name' : ['bone marrow', 'dentategyrus', 'pancreas']
}

2. Use this function to create a JSON file with a dictionary for each possible combination of these parameters.

In [ ]:
# export
import json
import os

counter = 0
def parameters_list_from_dict(parameters_dict, outfile):
    ''' Given a dictionary whose values are each a list of possible values, creates a json file with one dictionary for every possible combination of values. '''
    with open(outfile, 'a') as f:
                f.write('{')
    keys = list(parameters_dict.keys())
    run_num = 0
    def add_key_to_dict(little_dict, key_num):
        if key_num == len(keys):
            global counter
            counter += 1
            # write little dict to outfile
            descriptor = f'run_{counter}'
            little_dict['output_label'] = descriptor
            with open(outfile, 'a') as f:
                f.write(f'\n"{descriptor}":')
                json.dump(little_dict,f,indent=4)
                f.write(',')
        else:
            current_key = keys[key_num]
            for val in parameters_dict[current_key]:
                little_dict[current_key] = val
                add_key_to_dict(little_dict.copy(), key_num + 1)
    little_dict = {}
    add_key_to_dict(little_dict, 0)
    # delete last trailing comma, as json doesn't like this
    with open(outfile, 'rb+') as f:
        f.seek(-1, os.SEEK_END)
        f.truncate()
    with open(outfile, 'a') as f:
        f.write('}')
    print(f"Created {counter} test scenarios.")
            

In [ ]:
parameters_list_from_dict(parameters_dict, 'phate_single_cell_tests.json')

Created 3 test scenarios.
